In [60]:
from neo4j import GraphDatabase

class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__password = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

# Conectar a la base de datos
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="qwer1234")



Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


### Q1
Obtener los amigos y familiares de un usuario determinado.

In [61]:
nombre_usuario = "Juan_Alberto"
query = """
MATCH (p:Persona {nombre: $nombre})-[:FAMILIA|AMIGO]-(relacionado)
RETURN DISTINCT relacionado.nombre AS Nombre
"""
amigos_y_familiares = conn.query(query, parameters={'nombre': nombre_usuario})
for persona in amigos_y_familiares:
    print(persona["Nombre"])

Romeo_Santos
Belen
Dante
Elvira


### Q2
Obtener los familiares de los familiares de un usuario determinado.

In [62]:
query = """
MATCH (p:Persona {nombre: $nombre})-[:FAMILIA]->(familiar)-[:FAMILIA]->(familiar_de_familiar)
RETURN DISTINCT familiar_de_familiar.nombre AS Nombre
"""
familiares_de_familiares = conn.query(query, parameters={'nombre': nombre_usuario})
for persona in familiares_de_familiares:
    print(persona["Nombre"])

Estéfano
Julieta
Alessandra


### Q3
Obtener todos los mensajes enviados de un usuario determinado a otro usuario
determinado después de una fecha especificada.

In [63]:
nombre_remitente = 'Estéfano'
nombre_destinatario = 'Dante'
fecha_especificada = '2023-01-12T13:02:21Z'

query_mensajes_personalizada = """
MATCH (remitente:Persona {nombre: $nombre_remitente})-[conversa:CONVERSA]-(destinatario:Persona {nombre: $nombre_destinatario})
WHERE conversa.timestamp >= datetime($fecha_especificada)
RETURN conversa
"""

mensajes_despues_fecha = conn.query(query_mensajes_personalizada, parameters={
    'nombre_remitente': nombre_remitente,
    'nombre_destinatario': nombre_destinatario,
    'fecha_especificada': fecha_especificada
})

for conversa in mensajes_despues_fecha:
    print(f"Mensaje: {conversa['conversa']['mensaje']}, Fecha: {conversa['conversa']['timestamp']}")

Mensaje: Te iba a invitar a mi casa pero no pasa nada jaja, Fecha: 2023-01-12T13:02:21.000000000+00:00


### Q4
Obtener la conversación completa entre dos usuarios determinados.

In [64]:
nombre_remitente = "Estéfano"
nombre_destinatario = "Dante"

query_conversacion_completa = """
MATCH (remitente:Persona {nombre: $nombre_remitente})-[conversa:CONVERSA]-(destinatario:Persona {nombre: $nombre_destinatario})
RETURN conversa.fecha_conversacion AS Fecha, conversa.mensaje AS Mensaje
ORDER BY conversa.fecha_conversacion
"""

conversacion_completa = conn.query(query_conversacion_completa, parameters={
    'nombre_remitente': nombre_remitente,
    'nombre_destinatario': nombre_destinatario
})

for registro in conversacion_completa:
    mensaje = registro['Mensaje']
    print({mensaje})

{'Hey Dante, que tal estas?'}
{'Hola estefano, voy liadisimo con un trabajo de BBDD, tu que tal??'}
{'Te iba a invitar a mi casa pero no pasa nada jaja'}


### Q5
Obtener todos los usuarios mencionados por un usuario determinado los cuales
tengan una relación laboral con el usuario que los mencionó.

In [65]:
nombre_usuario = "Romeo_Santos"

query_usuarios_mencionados = """
MATCH (usuario:Persona {nombre: $nombre_usuario})-[:PUBLICA]->(publicacion:Publicacion)-[:MENCIONA]->(mencionado:Persona),
      (usuario)-[:TRABAJA|TRABAJABA]->(empresa:Empresa)<-[:TRABAJA|TRABAJABA]-(mencionado)
RETURN DISTINCT mencionado.nombre AS UsuarioMencionado
"""

usuarios_mencionados = conn.query(query_usuarios_mencionados, parameters={
    'nombre_usuario': nombre_usuario
})

for usuario in usuarios_mencionados:
    print(usuario['UsuarioMencionado'])

Estéfano
Alessandra


### Q6
Obtener los usuarios (terceros) que, no teniendo relación con un usuario
determinado (primero), tengan alguna relación en uno o varios saltos de relación
con los usuarios (segundos) que tienen relación con el usuario determinado. Se
podrá definir el número de saltos máximo en la consulta. En la consulta se
mostrará el usuario segundo del que parte la relación con los terceros y el
número de saltos de relación entre dichos usuarios y se ordenará por número de
saltos

In [66]:
nombre_usuario = "Luis"
numero_saltos = 3  # Al estar usando expresion regular para el num saltos, usamos f y {} en vez del $

query_usuarios_terceros = f"""
MATCH (usuario_primero:Persona {{nombre: '{nombre_usuario}'}})
MATCH camino=(usuario_primero)-[*{numero_saltos}]-(usuario_tercero:Persona)
WHERE NOT (usuario_primero)-[]-(usuario_tercero) AND usuario_primero <> usuario_tercero
RETURN DISTINCT usuario_tercero.nombre AS UsuarioTercero
"""

usuarios_terceros = conn.query(query_usuarios_terceros)

for registro in usuarios_terceros:
    print(f"Usuario Tercero: {registro['UsuarioTercero']}")

Usuario Tercero: Juan_Alberto
Usuario Tercero: Belen
Usuario Tercero: Elvira
Usuario Tercero: Orslok
Usuario Tercero: Romeo_Santos
Usuario Tercero: Julieta
Usuario Tercero: Estéfano
Usuario Tercero: Alessandra


### Q7
Obtener los usuarios (terceros) que, no teniendo relación con un usuario
determinado(primero), tengan alguna relación con los usuarios (segundos) que
tienen relación con el usuario determinado. Solo se mostrarán las relaciones
entre usuarios que tengan más de un número especificado de mensajes. Ordenar
el resultado primero por el número de mensajes entre el primer usuario y los
segundos y después por el número de mensajes entre segundos y los terceros

In [67]:
nombre_usuario = "Alessandra" 
numero_mensajes_minimo = 1  

query = f"""
MATCH (primero:Persona {{nombre: '{nombre_usuario}'}})-[r1:CONVERSA]-(segundo:Persona)
WITH primero, segundo, COUNT(r1) AS numMensajesPrimeroSegundo
WHERE numMensajesPrimeroSegundo > {numero_mensajes_minimo}
MATCH (segundo)-[r2:CONVERSA]-(tercero:Persona)
WHERE NOT (primero)-[:CONVERSA]-(tercero) AND primero <> tercero
RETURN segundo.nombre AS UsuarioSegundo, tercero.nombre AS UsuarioTercero, numMensajesPrimeroSegundo, COUNT(r2) AS numMensajesSegundoTercero
ORDER BY numMensajesPrimeroSegundo DESC, numMensajesSegundoTercero DESC
"""

resultados = conn.query(query)

for resultado in resultados:
    print(f"Usuario Segundo: {resultado['UsuarioSegundo']}\nUsuario Tercero: {resultado['UsuarioTercero']}\n"
          f"Número de Mensajes (Primero-Segundo): {resultado['numMensajesPrimeroSegundo']}\n"
          f"Número de Mensajes (Segundo-Tercero): {resultado['numMensajesSegundoTercero']}")

Usuario Segundo: Estéfano
Usuario Tercero: Dante
Número de Mensajes (Primero-Segundo): 2
Número de Mensajes (Segundo-Tercero): 3


In [68]:
conn.close()